In [1]:
import numpy as np
import lsst.sims.featureScheduler as fs
from lsst.sims.speedObservatory import Speed_observatory
import matplotlib.pylab as plt
import healpy as hp
from numpy.lib.recfunctions import append_fields
from lsst.sims.utils import m5_flat_sed, raDec2Hpid, Site, _hpid2RaDec

In [2]:
from lsst.sims.featureScheduler import features
from lsst.sims.featureScheduler import basis_functions
from scipy.spatial import cKDTree as kdtree
from scipy.stats import binned_statistic
from lsst.sims.featureScheduler.thomson import xyz2thetaphi, thetaphi2xyz
from lsst.sims.featureScheduler.surveys import rotx,empty_observation

In [3]:
from lsst.sims.featureScheduler import BaseSurvey

In [28]:
class Greedy_survey_fields_clouds(BaseSurvey):
    """
    Use a field tesselation and assign each healpix to a field.
    """
    def __init__(self, basis_functions, basis_weights, extra_features=None, filtername='r',
                 block_size=25, smoothing_kernel=None, nside=32,
                 dither=False, seed=42, ignore_obs='ack',
                 tag_fields=False, tag_map=None, tag_names=None):
        if extra_features is None:
            extra_features = {}
            extra_features['night'] = features.Current_night()
            extra_features['mounted_filters'] = features.Mounted_filters()
            extra_features['cloud'] = features.CloudsFeature(cloud_max=-0.1)
        if tag_fields and tag_names is not None:
            extra_features['proposals'] = features.SurveyProposals(ids=tag_names.keys(),
                                                                   names=tag_names.values())
        super(Greedy_survey_fields_clouds, self).__init__(basis_functions=basis_functions,
                                                   basis_weights=basis_weights,
                                                   extra_features=extra_features,
                                                   smoothing_kernel=smoothing_kernel,
                                                   ignore_obs=ignore_obs,
                                                   nside=nside)
        self.filtername = filtername
        self.block_size = block_size
        np.random.seed(seed)
        self.dither = dither
        self.night = extra_features['night'].feature + 0
        self.tag_map = tag_map
        self.tag_fields = tag_fields
        # self.inside_tagged = np.zeros_like(self.hp2fields) == 0

        if tag_fields:
            tags = np.unique(tag_map[tag_map > 0])
            for tag in tags:
                inside_tag = np.where(tag_map == tag)
                fields_id = np.unique(self.hp2fields[inside_tag])
                self.fields['tag'][fields_id] = tag
        else:
            for i in range(len(self.fields)):
                self.fields['tag'][i] = 1

    def _check_feasability(self):
        """
        Check if the survey is feasible in the current conditions
        """
        return self.filtername in self.extra_features['mounted_filters'].feature

    def _spin_fields(self, lon=None, lat=None):
        """Spin the field tesselation
        """
        if lon is None:
            lon = np.random.rand()*np.pi*2
        if lat is None:
            lat = np.random.rand()*np.pi*2
        # rotate longitude
        ra = (self.fields['RA'] + lon) % (2.*np.pi)
        dec = self.fields['dec'] + 0

        # Now to rotate ra and dec about the x-axis
        x, y, z = thetaphi2xyz(ra, dec+np.pi/2.)
        xp, yp, zp = rotx(lat, x, y, z)
        theta, phi = xyz2thetaphi(xp, yp, zp)
        dec = phi - np.pi/2
        ra = theta + np.pi

        self.fields['RA'] = ra
        self.fields['dec'] = dec
        # Rebuild the kdtree with the new positions
        # XXX-may be doing some ra,dec to conversions xyz more than needed.
        self._hp2fieldsetup(ra, dec)

    def update_conditions(self, conditions, **kwargs):
        for bf in self.basis_functions:
            bf.update_conditions(conditions, **kwargs)
        for feature in self.extra_features:
            if hasattr(self.extra_features[feature], 'update_conditions'):
                self.extra_features[feature].update_conditions(conditions, **kwargs)
        # If we are dithering and need to spin the fields
        if self.dither:
            if self.extra_features['night'].feature != self.night:
                self._spin_fields()
                self.night = self.extra_features['night'].feature + 0
        self.reward_checked = False

    def add_observation(self, observation, **kwargs):
        # ugh, I think here I have to assume observation is an array and not a dict.

        if self.ignore_obs not in observation['note']:
            for bf in self.basis_functions:
                bf.add_observation(observation, **kwargs)
            for feature in self.extra_features:
                if hasattr(self.extra_features[feature], 'add_observation'):
                    self.extra_features[feature].add_observation(observation, **kwargs)
            self.reward_checked = False

    def __call__(self):
        """
        Just point at the highest reward healpix
        """
        if not self.reward_checked:
            self.reward = self.calc_reward_function()
        # Let's find the best N from the fields
        order = np.argsort(self.reward)[::-1]

        iter = 0
        while True:
            best_hp = order[iter*self.block_size:(iter+1)*self.block_size]
            best_fields = np.unique(self.hp2fields[best_hp])
            observations = []
            for field in best_fields:
                if self.tag_fields:
                    tag = np.unique(self.tag_map[np.where(self.hp2fields == field)])[0]
                else:
                    tag = 1
                if tag == 0:
                    continue
                if self.extra_features['cloud'].feature < 1.0:
                    break
                else:
                    obs = empty_observation()
                    obs['RA'] = self.fields['RA'][field]
                    obs['dec'] = self.fields['dec'][field]
                    obs['rotSkyPos'] = 0.
                    obs['filter'] = self.filtername
                    obs['nexp'] = 2.  # FIXME: hardcoded
                    obs['exptime'] = 30.  # FIXME: hardcoded
                    obs['field_id'] = -1
                    if self.tag_fields:
                        obs['survey_id'] = np.unique(self.tag_map[np.where(self.hp2fields == field)])[0]
                    else:
                        obs['survey_id'] = 1

                    observations.append(obs)
                    break
            iter += 1
            if len(observations) > 0 or (iter+2)*self.block_size > len(order):
                break
                
        return observations

In [29]:
 if __name__ == '__main__':
    nside = fs.set_default_nside(nside=32)
 
    survey_length = 1.0  # days
 
    # Define what we want the final visit ratio map to look like

    target_maps = {}
    nside = fs.set_default_nside(nside=32)  # Required
    target_maps['r'] = fs.generate_goal_map(NES_fraction=0.46,
                                            WFD_fraction=1.0, SCP_fraction=0.15,
                                            GP_fraction=0.15, nside=nside,
                                            generate_id_map=True)

    years = np.round(survey_length/365.25)
    filters = ['r']
    surveys = []
 
    for filtername in filters:
        bfs = []
        bfs.append(fs.M5_diff_basis_function(filtername=filtername, nside=nside))
        bfs.append(fs.Target_map_basis_function(filtername=filtername,
                                                target_map=target_maps[filtername][1],
                                                out_of_bounds_val=hp.UNSEEN, nside=nside))
 
        bfs.append(fs.North_south_patch_basis_function(zenith_min_alt=50., nside=nside))
        bfs.append(fs.Slewtime_basis_function(filtername=filtername, nside=nside))
        bfs.append(fs.Strict_filter_basis_function(filtername=filtername))
 
        weights = np.array([3.0, 0.2, 1., 3., 3.])
        surveys.append(Greedy_survey_fields_clouds(bfs, weights, block_size=1, 
                                               filtername=filtername, tag_fields=True,
                                               tag_map = target_maps[filtername][1], 
                                               tag_names=target_maps[filtername][2],
                                               dither=True, nside=nside))
 
    scheduler = fs.Core_scheduler(surveys, nside=nside)
    observatory = Speed_observatory(nside=nside,quickTest=False,mjd_start=59580.035000 + 2)
    observatory, scheduler, observations = fs.sim_runner(observatory, scheduler,
                                                         survey_length=survey_length,
                                                         filename='full_nside32_fast_%i.db' % years,
                                                         delete_past=True)

None


IndexError: pop from empty list